In [32]:
using Serialization, Turing
include("ode_problem.jl");
include("target_probability.jl");
include("bayesian_inference.jl");
include("calculate_experimental_outputs.jl");

### Define Target Distribution for Sampling

In [33]:
#first input we need are data points and each point's associated standard deviation to evaluate the likelihood
data_points = deserialize("outputs/000_all_training_data.dict")["response"]
std_dev = deserialize("outputs/000_all_training_data.dict")["average_error"]

#next input we need is an ode problem
egf_dosage = 20
odesys, u0, tspan, p = return_ode_problem_default_inputs(egf_dosage);
odeprob = DifferentialEquations.ODEProblem(odesys, u0, tspan, p);

#and ode solver inputs, which are consistent for all simulations
solver_inputs_timecourse = return_ode_problem_solver_default_inputs()

#also need regularization hyperparameters
regularization_hyperparams = deserialize("outputs/000_regularization_parameters_predicted.dict")
regularization_hyperparams_theoretical = deserialize("outputs/000_regularization_parameters_reported.dict")

#finally, load initial conditions with different EGF ligand dosages for simulation
ligand_dosages = return_ligand_dose_order_for_likelihood()
initial_conditions = [return_ode_problem_default_inputs(egf_dosage)[2] for egf_dosage in ligand_dosages];

┌ Warning: Cannot separate bidirectional kineticLaw `(-k21b*ShG(t)) / default_compartment + (k21f*ShP(t)*Grb(t)) / (default_compartment^2)` to forward and reverse part. Setting forward to `(-k21b*ShG(t)) / default_compartment + (k21f*ShP(t)*Grb(t)) / (default_compartment^2)` and reverse to `0`. Stochastic simulations will be inexact.
└ @ SBMLToolkit ~/.julia/packages/SBMLToolkit/7C24E/src/reactions.jl:35
┌ Warning: Cannot separate bidirectional kineticLaw `(V4*RP(t)) / ((K4 + RP(t) / default_compartment)*default_compartment)` to forward and reverse part. Setting forward to `(V4*RP(t)) / ((K4 + RP(t) / default_compartment)*default_compartment)` and reverse to `0`. Stochastic simulations will be inexact.
└ @ SBMLToolkit ~/.julia/packages/SBMLToolkit/7C24E/src/reactions.jl:35
┌ Warning: Cannot separate bidirectional kineticLaw `(k20f*RShGS(t)) / default_compartment + (-k20b*RP(t)*ShGS(t)) / (default_compartment^2)` to forward and reverse part. Setting forward to `(k20f*RShGS(t)) / default

In [34]:
#target probability definition
target_distribution_regularized = logprob_regularized(data_points, odeprob, std_dev, regularization_hyperparams, solver_inputs_timecourse, initial_conditions, ligand_dosages);

target_distribution_regularized_theoretical = logprob_regularized(data_points, odeprob, std_dev, regularization_hyperparams_theoretical, solver_inputs_timecourse, initial_conditions, ligand_dosages);

target_distribution_unregularized = logprob_unregularized(data_points, odeprob, std_dev, solver_inputs_timecourse, initial_conditions, ligand_dosages);

### Run Affine Invariant Sampler

In [35]:
#affine invariant sampler parameters
n_ensemble = 1
n_walkers = 5
n_iterations = 5

#target probability
target_probability = Dict("unregularized"=>target_distribution_unregularized, 
"regularized"=>target_distribution_regularized, "regularized_theoretical"=>target_distribution_regularized_theoretical)

Dict{String, Model{F, argnames, (), (), Targs, Tuple{}, DefaultContext} where {F, argnames, Targs}} with 3 entries:
  "regularized"             => Model{typeof(logprob_regularized), (:data, :odep…
  "regularized_theoretical" => Model{typeof(logprob_regularized), (:data, :odep…
  "unregularized"           => Model{typeof(logprob_unregularized), (:data, :od…

In [37]:
#initial run for both regularized and unregularized 
for i in ["unregularized", "regularized","regularized_theoretical"]
    affine_invariant_mcmc_firstrun(n_ensemble, n_walkers, n_iterations, target_probability[i], i)
end

In [ ]:
start = 1
n_subchains = 1
approach = "regularized"
for i in start:n_subchains
    affine_invariant_mcmc(n_ensemble, n_walkers, n_iterations, target_probability[approach], i, approach)
end

approach = "unregularized"
for i in start:n_subchains
    affine_invariant_mcmc(n_ensemble, n_walkers, n_iterations, target_probability[approach], i, approach)
end

approach = "regularized_theoretical"
for i in start:n_subchains
    affine_invariant_mcmc(n_ensemble, n_walkers, n_iterations, target_probability[approach], i, approach)
end

┌ Warning: At t=242.32597937885015, dt was forced below floating point epsilon 2.842170943040401e-14, and step error estimate = 1.1174328132205966. Aborting. There is either an error in your model specification or the true solution is unstable (or the true solution can not be represented in the precision of Float64).
└ @ SciMLBase ~/.julia/packages/SciMLBase/hJh6T/src/integrator_interface.jl:623
┌ Warning: At t=277.64108376658334, dt was forced below floating point epsilon 5.684341886080802e-14, and step error estimate = 1.117433011564668. Aborting. There is either an error in your model specification or the true solution is unstable (or the true solution can not be represented in the precision of Float64).
└ @ SciMLBase ~/.julia/packages/SciMLBase/hJh6T/src/integrator_interface.jl:623
┌ Warning: At t=391.6874263898566, dt was forced below floating point epsilon 5.684341886080802e-14, and step error estimate = 1.1174328085906997. Aborting. There is either an error in your model specifi